In [5]:
# install packages
!pip install ipyleaflet
!pip install pandas
!pip install geopandas
!pip install shapely
!pip install notebook

In [ ]:
# install & enable aditional package for enabling interactive map within the notebook
!jupyter nbextension install ipyleaflet --user --py
!jupyter nbextension enable ipyleaflet --user --py

In [20]:
# import packages
import ipyleaflet
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

In [17]:
# create mapping widget
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, projections
watercolor = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
m = Map(layers=(watercolor, ), center=(48, -123), zoom=3) 
draw_control = DrawControl()

draw_control.circle = {
    "shapeOptions": {
        "fillColor": "#efed69",
        "color": "#efed69",
        "fillOpacity": 1.0
    }
}

feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""    
    feature_collection['features'].append(geo_json)

draw_control.on_draw(handle_draw)

m.add_control(draw_control)
m

Map(center=[48, -123], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [16]:
# create mapping widget with sattelite map
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, projections

# switch the base map here
watercolor = basemap_to_tiles(basemaps.Gaode.Satellite)
m = Map(layers=(watercolor, ), center=(48, -123), zoom=3) 
draw_control = DrawControl()

# change mapping tool UI
draw_control.circle = {
    "shapeOptions": {
        "fillColor": "#efed69",
        "color": "#efed69",
        "fillOpacity": 1.0
    }
}

# save map features to a dictionary
feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""    
    feature_collection['features'].append(geo_json)

draw_control.on_draw(handle_draw)

m.add_control(draw_control)
m

Map(center=[48, -123], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [18]:
# print out the dictionary of features you drew on the map
print(feature_collection)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'style': {'stroke': True, 'color': '#3388ff', 'weight': 4, 'opacity': 0.5, 'fill': True, 'fillColor': None, 'fillOpacity': 0.2, 'clickable': True}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-87.650717, 41.880358], [-87.649763, 41.880382], [-87.649704, 41.879235], [-87.650701, 41.879215], [-87.650717, 41.880358]]]}}, {'type': 'Feature', 'properties': {'style': {'stroke': True, 'color': '#3388ff', 'weight': 4, 'opacity': 0.5, 'fill': True, 'fillColor': None, 'fillOpacity': 0.2, 'clickable': True}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-87.649559, 41.88037], [-87.648625, 41.880382], [-87.648604, 41.880114], [-87.649516, 41.88011], [-87.649559, 41.88037]]]}}]}


In [21]:
# parse dictionary of features into shapes
parsed_data = [[item['geometry']['type'],
                 Polygon(item['geometry']['coordinates'][0])] for item in feature_collection['features']]
shapes=gpd.GeoDataFrame(data=parsed_data, columns=['type', 'geometry'], crs="EPSG:4326")
shapes

,type,geometry
0,Polygon,"POLYGON ((-87.65072 41.88036, -87.64976 41.880..."
1,Polygon,"POLYGON ((-87.64956 41.88037, -87.64862 41.880..."


In [22]:
shapes.area

/Users/claudianikel/Desktop/mapping_project/my_virtual_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


0    1.117773e-06
1    2.439280e-07
dtype: float64

In [23]:
# convert sqm into sqft
shapes = shapes.to_crs(6933)
shapes['area_sqft_espg_6933']=shapes.area*10.7639

shapes = shapes.to_crs(32633)
shapes['area_sqft_espg_32633']=shapes.area*10.7639

shapes = shapes.to_crs(3857)
shapes['area_sqft_espg_3857']=shapes.area*10.7639

shapes = shapes.to_crs(26915)
shapes['area_sqft_espg_26915']=shapes.area*10.7639



#convert back to ESP4326 (this is what Google Maps uses)
shapes = shapes.to_crs(4326)
shapes

,type,geometry,area_sqft_espg_6933,area_sqft_espg_32633,area_sqft_espg_3857,area_sqft_espg_26915
0,Polygon,"POLYGON ((-87.65072 41.88036, -87.64976 41.880...",110926.592371,233932.260524,200250.400190,111376.584360
1,Polygon,"POLYGON ((-87.64956 41.88037, -87.64863 41.880...",24206.996991,51049.670216,43700.324504,24305.246278
